# DeepLake + LlamaIndex

Look at financial statements

In [1]:
!pip install llama-index deeplake

You should consider upgrading via the '/Users/adilkhansarsen/Documents/work/LlamaIndex/llama_index/GPTIndex/bin/python3 -m pip install --upgrade pip' command.


In [1]:
# My OpenAI Key
import os
import getpass
os.environ['OPENAI_API_KEY'] = getpass.getpass("OpenAI token: ")

In [2]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
from llama_index import (
    VectorStoreIndex,
    SimpleKeywordTableIndex, 
    SimpleDirectoryReader,
    LLMPredictor,
    ServiceContext,
    download_loader,
    Document,
)
from llama_index.vector_stores import DeepLakeVectorStore
from langchain.llms.openai import OpenAIChat
from typing import List, Optional, Tuple
from pathlib import Path
import requests
import tqdm

INFO:numexpr.utils:Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


/Users/adilkhansarsen/Documents/work/LlamaIndex/llama_index/GPTIndex/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Ingest Data (PDFs of Financial Statements)

In [4]:
# financial reports of amamzon, but can be replaced by any URLs of pdfs
urls = ['https://s2.q4cdn.com/299287126/files/doc_financials/Q1_2018_-_8-K_Press_Release_FILED.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/Q2_2018_Earnings_Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_news/archive/Q318-Amazon-Earnings-Press-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_news/archive/AMAZON.COM-ANNOUNCES-FOURTH-QUARTER-SALES-UP-20-TO-$72.4-BILLION.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/Q119_Amazon_Earnings_Press_Release_FINAL.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_news/archive/Amazon-Q2-2019-Earnings-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_news/archive/Q3-2019-Amazon-Financial-Results.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_news/archive/Amazon-Q4-2019-Earnings-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/2020/Q1/AMZN-Q1-2020-Earnings-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/2020/q2/Q2-2020-Amazon-Earnings-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/2020/q4/Amazon-Q4-2020-Earnings-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/2021/q1/Amazon-Q1-2021-Earnings-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/2021/q2/AMZN-Q2-2021-Earnings-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/2021/q3/Q3-2021-Earnings-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/2021/q4/business_and_financial_update.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/2022/q1/Q1-2022-Amazon-Earnings-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/2022/q2/Q2-2022-Amazon-Earnings-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/2022/q3/Q3-2022-Amazon-Earnings-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/2022/q4/Q4-2022-Amazon-Earnings-Release.pdf'
    ]

# hardcoding for now since we're missing q3 2020
years = [
    2018, 2018, 2018, 2018,
    2019, 2019, 2019, 2019,
    2020, 2020, 2020,
    2021, 2021, 2021, 2021,
    2022, 2022, 2022, 2022
]
months = [
    1, 4, 7, 10,
    1, 4, 7, 10,
    1, 4, 10,
    1, 4, 7, 10,
    1, 4, 7, 10
]

zipped_data = list(zip(urls, months, years))

In [5]:
PDFReader = download_loader("PDFReader")

loader = PDFReader()

In [6]:
def download_reports(data: List[Tuple[str, int, int]], out_dir: Optional[str] = None) -> List[Document]:
    """Download pages from a list of urls."""
    docs = []
    out_dir = Path(out_dir or ".")
    if not out_dir.exists():
        print(out_dir)
        os.makedirs(out_dir)

    
    for url, month, year in tqdm.tqdm(data):
        path_base = url.split('/')[-1]
        out_path = out_dir / path_base
        if not out_path.exists():
            r = requests.get(url)
            with open(out_path, 'wb') as f:
                f.write(r.content)
        doc = loader.load_data(file=Path(out_path))[0]
        
        date_str = f"{month:02d}" + "-01-" + str(year)
        doc.extra_info = {"Date": date_str}
        
        docs.append(doc)
    return docs

def _get_quarter_from_month(month: int) -> str:
    mapping = {
        1: "Q1",
        4: "Q2",
        7: "Q3",
        10: "Q4"
    }
    return mapping[month]

In [7]:
docs = download_reports(zipped_data, 'data')

100%|██████████| 19/19 [00:13<00:00,  1.44it/s]


### Build Vector Indices

In [8]:
llm_predictor_chatgpt = LLMPredictor(
    llm=OpenAIChat(temperature=0, model_name="gpt-3.5-turbo-16k-0613")
)
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor_chatgpt)

/Users/adilkhansarsen/Documents/work/LlamaIndex/llama_index/GPTIndex/lib/python3.9/site-packages/langchain/llms/openai.py:769: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [9]:
# Build city document index
from llama_index.storage.storage_context import StorageContext

# build vector index for each quarterly statement, store in dictionary
dataset_root = 'amazon_example/amazon_financial_'
vector_indices = {}
for idx, (_, month, year) in enumerate(zipped_data):
    doc = docs[idx]

    dataset_path = dataset_root + f"{month:02d}_{year}"
    vector_store = DeepLakeVectorStore(
        dataset_path=dataset_path,
        overwrite=True,
        verbose=False,
    ) 
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
        
    vector_index = VectorStoreIndex.from_documents([doc], storage_context=storage_context, service_context=service_context)
    vector_indices[(month, year)] = vector_index

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 1023 tokens
> [build_index_from_nodes] Total embedding token usage: 1023 tokens


INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 1118 tokens
> [build_index_from_nodes] Total embedding token usage: 1118 tokens


INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 917 tokens
> [build_index_from_nodes] Total embedding token usage: 917 tokens


INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 793 tokens
> [build_index_from_nodes] Total embedding token usage: 793 tokens


INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 927 tokens
> [build_index_from_nodes] Total embedding token usage: 927 tokens


...

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 842 tokens
> [build_index_from_nodes] Total embedding token usage: 842 tokens


INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 842 tokens
> [build_index_from_nodes] Total embedding token usage: 842 tokens


INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 839 tokens
> [build_index_from_nodes] Total embedding token usage: 839 tokens


INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 638 tokens
> [build_index_from_nodes] Total embedding token usage: 638 tokens


INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 701 tokens
> [build_index_from_nodes] Total embedding token usage: 701 tokens


#### Test Querying a Vector Index

In [10]:
response = vector_indices[(1, 2018)].as_query_engine(service_context=service_context).query("What is the operating cash flow?")

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 7 tokens
> [retrieve] Total embedding token usage: 7 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1081 tokens
> [get_response] Total LLM token usage: 1081 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


In [11]:
print(str(response))
print(response.get_formatted_sources())

The operating cash flow for the trailing twelve months ended March 31, 2018, was $18.2 billion.
> Source (Doc id: e764aa30-7451-4c93-aac3-402bb1dd7aba): 1 
   
AMAZON.COM ANNOUNCES FIRST QUARTER SALES UP 43% TO $51.0 BILLION  
SEATTLE —(BUSINESS WIRE...

> Source (Doc id: 934a2360-2fa6-4fbc-9706-2e0cc743d9be): and Insignia brands, available for purchase in 2018 through Best Buy stores, 
BestBuy.com, and Am...


In [12]:
response = vector_indices[(1, 2018)].as_query_engine(service_context=service_context).query("What are the updates on Whole Foods?")

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 8 tokens
> [retrieve] Total embedding token usage: 8 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1073 tokens
> [get_response] Total LLM token usage: 1073 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


In [13]:
print(response)

The given context information does not provide any updates on Whole Foods.


### Build Graph: Keyword Table Index on top of vector indices! 

We compose a keyword table index on top of all the vector indices.

In [14]:
from llama_index.indices.composability.graph import ComposableGraph

In [15]:
# set summary text for city
index_summaries = {}
for idx, (_, month, year) in enumerate(zipped_data):
    quarter_str = _get_quarter_from_month(month)
    index_summaries[(month, year)] = f"Amazon Financial Statement, {quarter_str}, {year}"

In [16]:
graph = ComposableGraph.from_indices(
    SimpleKeywordTableIndex,
    [index for _, index in vector_indices.items()], 
    [summary for _, summary in index_summaries.items()],
    max_keywords_per_chunk=50
)

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 0 tokens
> [build_index_from_nodes] Total embedding token usage: 0 tokens


In [17]:
from llama_index.indices.query.query_transform.base import DecomposeQueryTransform
decompose_transform = DecomposeQueryTransform(
    llm_predictor_chatgpt, 
    verbose=True
)

In [18]:
# TMP 
query_str = "Analyze revenue in Q1 of 2018."

In [19]:
# with query decomposition in subindices
from llama_index.query_engine.transform_query_engine import TransformQueryEngine


custom_query_engines = {}
for index in vector_indices.values():
    query_engine = index.as_query_engine(service_context=service_context)
    transform_metadata = {'index_summary': index.index_struct.summary}
    tranformed_query_engine = TransformQueryEngine(query_engine, decompose_transform, transform_metadata=transform_metadata)
    custom_query_engines[index.index_id] = tranformed_query_engine

custom_query_engines[graph.root_index.index_id] = graph.root_index.as_query_engine(
    retriever_mode='simple', 
    response_mode='tree_summarize', 
    service_context=service_context
)

query_engine_decompose = graph.as_query_engine(
    custom_query_engines=custom_query_engines,
)

In [20]:
from llama_index.indices.query.query_transform.base import DecomposeQueryTransform
decompose_transform = DecomposeQueryTransform(
    llm_predictor_chatgpt, verbose=True
)

In [23]:
response_chatgpt = query_engine_decompose.query(
    "Analyze revenue in Q1 of 2018."
)

INFO:llama_index.indices.keyword_table.retrievers:> Starting query: Analyze revenue in Q1 of 2018.
> Starting query: Analyze revenue in Q1 of 2018.
INFO:llama_index.indices.keyword_table.retrievers:query keywords: ['revenue', '2018', 'q1', 'analyze']
query keywords: ['revenue', '2018', 'q1', 'analyze']
INFO:llama_index.indices.keyword_table.retrievers:> Extracted keywords: ['2018', 'q1']
> Extracted keywords: ['2018', 'q1']
> Current query: Analyze revenue in Q1 of 2018.
> New query: What was the total revenue of Amazon in Q1 of 2018?
INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 13 tokens
> [retrieve] Total embedding token usage: 13 tokens
> Current query: Analyze revenue in Q1 of 2018.
> New query: What was the total revenue of Amazon in Q1 of 2018?
INFO:llama_index.token_counter.token_counter:> [get_response] 

In [24]:
print(str(response_chatgpt))

Based on the given context information, the revenue of Amazon in Q1 of 2018 was $51.0 billion.


In [79]:
response_chatgpt = query_engine_decompose.query(
    "Analyze revenue in Q2 of 2018."
)

INFO:llama_index.indices.keyword_table.retrievers:> Starting query: Analyze revenue in Q2 of 2018.
> Starting query: Analyze revenue in Q2 of 2018.
INFO:llama_index.indices.keyword_table.retrievers:query keywords: ['2018', 'revenue', 'analyze', 'q2']
query keywords: ['2018', 'revenue', 'analyze', 'q2']
INFO:llama_index.indices.keyword_table.retrievers:> Extracted keywords: ['2018', 'q2']
> Extracted keywords: ['2018', 'q2']
> Current query: Analyze revenue in Q2 of 2018.
> New query: What was the total revenue of Amazon in Q2 of 2018?
INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 13 tokens
> [retrieve] Total embedding token usage: 13 tokens
> Current query: Analyze revenue in Q2 of 2018.
> New query: What was the total revenue of Amazon in Q2 of 2018?
INFO:llama_index.token_counter.token_counter:> [get_response] 

In [80]:
print(str(response_chatgpt))

Based on the given context information, the revenue of Amazon in Q2 of 2018 was $52.9 billion.


In [98]:
response_chatgpt = query_engine_decompose.query(
    "Analyze and comapre revenue in Q1 and Q2 of 2018."
)

INFO:llama_index.indices.keyword_table.retrievers:> Starting query: Analyze and comapre revenue in Q1 and Q2 of 2018.
> Starting query: Analyze and comapre revenue in Q1 and Q2 of 2018.
INFO:llama_index.indices.keyword_table.retrievers:query keywords: ['comapre', 'q1', 'revenue', 'analyze', '2018', 'q2']
query keywords: ['comapre', 'q1', 'revenue', 'analyze', '2018', 'q2']
INFO:llama_index.indices.keyword_table.retrievers:> Extracted keywords: ['q1', '2018', 'q2']
> Extracted keywords: ['q1', '2018', 'q2']
> Current query: Analyze and comapre revenue in Q1 and Q2 of 2018.
> New query: What was the revenue of Amazon in Q1 of 2018?
INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 12 tokens
> [retrieve] Total embedding token usage: 12 tokens
> Current query: Analyze and comapre revenue in Q1 and Q2 of 2018.
> New query

In [99]:
print(str(response_chatgpt))

Based on the given context information, we can analyze and compare the revenue in Q1 and Q2 of 2018 for Amazon. 

The revenue of Amazon in Q1 of 2018 was $51.0 billion, while the revenue in Q2 of 2018 was $52.9 billion. Therefore, the revenue in Q2 of 2018 was higher than the revenue in Q1 of 2018. The difference between the two quarters is $1.9 billion.
